<h1>Import Functions</h1>

In [57]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

<h1>Load Data</h1>

In [30]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')  

<h1>Load Column Headers</h1>

In [119]:
headers = []
i=0
for th in soup.find_all('th'):
    if i < 3:
        headers.insert(i, th.text.replace('\n','') )
    i +=1
#df = pd.DataFrame(columns=headers)
#print(df)

Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []


<h1>Load Data into DataFrame</h1>

In [234]:
html_rows = soup.find('tbody')
y=0
out_rows = []
for tr in html_rows.find_all('tr'):
    x=0
    html_cols = []
    for td in tr.find_all('td'):
        html_cols.insert(x,td.text.replace('\n',''))
        x +=1
    if len(html_cols) > 0:
        #check if neighbourhood not assigned
        if html_cols[2] == 'Not assigned':
            html_cols[2] = html_cols[1]
        # are we on the first record?
        if y > 0 :
            # make sure the previous record exists
            if len(out_rows[y-1]) > 0:
                # check for same postal code
                if out_rows[y-1][0] == html_cols[0]:
                    out_rows[y-1][2] = out_rows[y-1][2] + ', ' + html_cols[2]
                else:
                    out_rows.insert(y,html_cols)
                    y +=1
            else:
                out_rows.insert(y,html_cols)
                y +=1
        else:
            out_rows.insert(y,html_cols)
            y +=1
df = pd.DataFrame(out_rows)
df.columns=headers
df = df[df.Borough != 'Not assigned']
print(df.head(10))

   Postcode           Borough                     Neighbourhood
2       M3A        North York                         Parkwoods
3       M4A        North York                  Victoria Village
4       M5A  Downtown Toronto         Harbourfront, Regent Park
5       M6A        North York  Lawrence Heights, Lawrence Manor
6       M7A      Queen's Park                      Queen's Park
8       M9A         Etobicoke                  Islington Avenue
9       M1B       Scarborough                    Rouge, Malvern
11      M3B        North York                   Don Mills North
12      M4B         East York   Woodbine Gardens, Parkview Hill
13      M5B  Downtown Toronto          Ryerson, Garden District


<h1>View the DataFrame Shape</h1>

In [235]:
df.shape

(103, 3)